In [ ]:
import requests
from bs4 import BeautifulSoup as be

import enum


class City(enum.Enum):
    Mondstadt = 0,
    Liyue = 1,
    Inazuma = 2,
    Sumeru = 3,
    Fontaine = 4,
    Natlan = 5,
    Snezhnaya = 6,


url_zh_channel_id = [150, 151, 324, 350]
url_zh_channel_id_all = 152

url_en_channel_id = [487, 488, 1108, 1148]
url_en_channel_id_all = 489

url_zh_prefix = "https://ys.mihoyo.com/content/ysCn/getContentList?"
url_page_size = 2000
url_page_num = 1
url_order = "asc"

url_en_prefix = "https://content-static-sea.hoyoverse.com/content/yuanshen/getContentList?"

def url_compose(prefix, channel_id, 
    page_size = url_page_size, page_num = url_page_num, order = url_order):
    url_res = prefix + \
        "pageSize=" + str(page_size) + \
        "&pageNum=" + str(page_num) + \
        "&order=" + order + \
        "&channelId=" + str(channel_id)
    print(url_res)
    return url_res

zh_url = [
    url_compose(url_zh_prefix, url_zh_channel_id[City.Mondstadt.value[0]]),
    url_compose(url_zh_prefix, url_zh_channel_id[City.Liyue.value[0]]),
    url_compose(url_zh_prefix, url_zh_channel_id[City.Inazuma.value[0]]),
    url_compose(url_zh_prefix, url_zh_channel_id[City.Sumeru.value[0]]),
    # url_compose(url_zh_prefix, url_zh_channel_id[City.Fontaine.value[0]]),
    # url_compose(url_zh_prefix, url_zh_channel_id[City.Natlan.value[0]]),
    # url_compose(url_zh_prefix, url_zh_channel_id[City.Snezhnaya.value[0]]),
]
en_url = [
    url_compose(url_en_prefix, url_en_channel_id[City.Mondstadt.value[0]]), 
    url_compose(url_en_prefix, url_en_channel_id[City.Liyue.value[0]]),
    url_compose(url_en_prefix, url_en_channel_id[City.Inazuma.value[0]]),
    url_compose(url_en_prefix, url_en_channel_id[City.Sumeru.value[0]]),
    # url_compose(url_en_prefix, url_en_channel_id[City.Fontaine.value[0]]),
    # url_compose(url_en_prefix, url_en_channel_id[City.Natlan.value[0]]),
    # url_compose(url_en_prefix, url_en_channel_id[City.Snezhnaya.value[0]]),
]

def get_json(_url_):
    req = requests.get(url=_url_)
    if req.status_code == 200:
        return req.json()['data']
    else:
        return None


def clean_data(_data_):
    _return_ = []
    for key in _data_['list']:
        ext = key["ext"]
        all_data = {key['title']: {
            "角色ICON": ext[0]["value"][0]["url"],
            "电脑端立绘": ext[1]["value"][0]["url"],
            "手机端立绘": ext[15]["value"][0]["url"],
            "角色名字": key['title'],
            "角色属性": ext[3]["value"][0]["url"],
            "角色语言": ext[4]["value"],
            "声优1": ext[5]["value"],
            "声优2": ext[6]["value"],
            "简介": be(ext[7]["value"], "lxml").p.text.strip(),
            "台词": ext[8]["value"][0]["url"],
            "音频": {
                ext[9]["value"][0]["name"]: ext[9]["value"][0]["url"],
                ext[10]["value"][0]["name"]: ext[10]["value"][0]["url"],
                ext[11]["value"][0]["name"]: ext[11]["value"][0]["url"],
                ext[12]["value"][0]["name"]: ext[12]["value"][0]["url"],
                ext[13]["value"][0]["name"]: ext[13]["value"][0]["url"],
                ext[14]["value"][0]["name"]: ext[14]["value"][0]["url"],
            },
        }
        }
        _return_.append(all_data[key['title']])
    return _return_


def data(lang):
    _json_ = {}
    for url in lang:
        jsonlist = clean_data(get_json(url))
        for json in jsonlist:
            _json_[json['角色名字']] = json
    print(_json_)
    return _json_


def lookup(name):
    json = data(zh_url)[name]
    print("查找角色：", name)
    for key, value in json.items():
        if key == "音频":
            for keys, values in json[key].items():
                print(f"{keys}：{values}")
        else:
            print(f"{key}：{value}")


if __name__ == '__main__':
    data(zh_url)
    data(en_url)
